In [2]:
# init
from os import path
import numpy as np
from pynput import keyboard
from pedalboard import Pedalboard, Chorus, Reverb
from pedalboard.io import AudioFile
from keyboard_layout import KeyboardLayout
from tone import Tone
from audio_player import AudioPlayer
from time_going import Timeline

smplRate = 44100

board = Pedalboard([Chorus(), Reverb(room_size=0.25)])

keyboard_layout = KeyboardLayout()
keyboard_layout.load('complete left to right')


complete left to right loaded as keyboard layout successfully.


In [3]:
tonemaker_1 = Tone(harmonics = [(2,2),(2.4,2.2),(2.7,2.6),(3.3,3),(4,4)])
def scaling_1(point):
    return 40 * 2**(point/12)
def scaling_2(point):
    scale = [2,2,1,2,2,2,1]
    result = 1
    for i in range(point):
        result += scale[i%len(scale)]
    return result

In [4]:
timeline = Timeline(IS_LOOPED = True)

In [6]:
audioplayer = AudioPlayer(timeline,board)
audioplayer.start()

# listen to keyboard
def on_press(key):
    try:
        fundamental = scaling_1(scaling_2(keyboard_layout.value.index(key.char)))
    except AttributeError:
        # print('special key {0} pressed'.format(
        #     key))
        return
    position = audioplayer.position
    tone = tonemaker_1.make(fundamental)/10
    data_size = tone.size
    if timeline.values.size >= (position + data_size):
        timeline.values[position:position+data_size] += tone
    else:
        tone.size - timeline.values.size
        timeline.values[position:] += tone[:timeline.values.size-position]
        timeline.values[:position+tone.size-timeline.values.size] += tone[timeline.values.size-position:]
    
def on_release(key):
    if key == keyboard.Key.esc:
        # Stop listener
        audioplayer.stop()
        print("stoped.")
        return False

# Collect events until released
# with keyboard.Listener(
#         on_press=on_press,
#         on_release=on_release) as listener:
#     listener.join()

# ...or, in a non-blocking fashion:
listener = keyboard.Listener(
    on_press=on_press,
    on_release=on_release)
listener.start()
print("started...")

started...


This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.


stoped.


In [6]:
# closing audio in case of emergency
audioplayer.stop()

In [6]:
timeline.clear()

In [5]:
timeline.save()

didn't save.


stoped.


In [5]:
timeline.load()

test1 loaded as timeline successfually.
